# Cook It Up with the Fantastic Four

In [33]:
# special IPython command to prepare the notebook for matplotlib
%matplotlib inline 

import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn
import statsmodels.api as sm

import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

# special matplotlib argument for improved plots
from matplotlib import rcParams

## Data Collection

In [34]:
full_df = pd.read_json("train.json")
full_df.head()

,cuisine,id,ingredients
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,22213,"[water, vegetable oil, wheat, salt]"
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe..."


In [35]:
test_df = pd.read_json("test.json")
test_df.head()

,id,ingredients
0,18009,"[baking powder, eggs, all-purpose flour, raisi..."
1,28583,"[sugar, egg yolks, corn starch, cream of tarta..."
2,41580,"[sausage links, fennel bulb, fronds, olive oil..."
3,29752,"[meat cuts, file powder, smoked sausage, okra,..."
4,35687,"[ground black pepper, salt, sausage casings, l..."


### Validation: Holdout Method

In a supervised learning problem one wants to determine how good the trained model is. Before the model is trained, the data is split into a training data set and a test data set. The training data set is used to train the model. The test data set is kept apart and used to determine how good the model is.

In [4]:
#This function returns a list of 100 id's for each cusine from input dataframe.
def build_df(indf):
    
    validdf_ids=[]
    random_ids=[]
    
    for k, v in indf.groupby('cuisine'):
        l=[]
        l.append([e for e in v.id])
        for i in xrange(len(l)):
            random_ids.append(np.random.choice(l[i], size=100, replace=False))
    
    validdf_ids = [item for sublist in random_ids for item in sublist]
    
    return validdf_ids

In [5]:
validation_df=full_df[full_df.id.isin(build_df(full_df))].reset_index()
train_df=full_df[~full_df.id.isin(build_df(full_df))].reset_index()

In [6]:
print "Full Dataframe"
print full_df.count()
print
print "Validation Dataframe"
print validation_df.count()
print
print "Training Dataframe"
print train_df.count()

Full Dataframe
cuisine        39774
id             39774
ingredients    39774
dtype: int64

Validation Dataframe
index          2000
cuisine        2000
id             2000
ingredients    2000
dtype: int64

Training Dataframe
index          37774
cuisine        37774
id             37774
ingredients    37774
dtype: int64


### Ingredients Map

In [7]:
#list of all possible ingredients

ingredients_all=[]

for i in full_df['ingredients']:
    for j in xrange(len(i)):
        ingredients_all.append(i[j])
        
for i in test_df['ingredients']:
    for j in xrange(len(i)):
        ingredients_all.append(i[j])

features=np.unique(ingredients_all) #create a feature list of unique ingrediants.

ingredients_map={v:k for k,v in enumerate(np.unique(ingredients_all).tolist())} #assign key to each ingredient

### Term-Document Matrix

In [8]:
def build_designmatrix(indf):
    
    designmatrix=np.zeros((len(indf), len(features))) 
    all_ids = indf['id'] #list of all id's from input set.
    
    #build design matrix
    for i in xrange(len(indf)):
        for j in indf['ingredients'][indf['id']==all_ids[i]]:
                for k in xrange(len(j)):
                        designmatrix[i, ingredients_map[j[k]]] = 1
                        
    return designmatrix

## Modeling & Optimization

### Decision Tree Classification Model

A decision tree aims at minimizing entropy. Entropy is a measure of chaos; something that is very ordered has a very low entropy, something that is very messy has a very high entropy. A decision tree is structure that resembles a flow chart. Every node in the tree represents a decision that needs to be taken for determining the class. Based on the training data the most relevant features and their values are selected for lowering the entropy of the data set. In contrast to many other approaches to classification, decision trees are easy to interpret by humans. Though decision trees are mainly used for classification, they can also be used for regression.

In [9]:
%%time
from sklearn import tree

clf = tree.DecisionTreeClassifier()
clf = clf.fit(build_designmatrix(train_df), train_df['cuisine'])
clf_predict=clf.predict(build_designmatrix(validation_df))

CPU times: user 5min 19s, sys: 3.53 s, total: 5min 23s
Wall time: 5min 24s


In [10]:
from sklearn.metrics import accuracy_score
print "Accuracy score for DecisionTreeClassifier" 
print accuracy_score(np.array(validation_df['cuisine']),clf_predict)

Accuracy score for DecisionTreeClassifier
0.931


In [11]:
predictions_as_dataframe = test_df.join(pd.DataFrame({"Prediction DT": clf_predict}))
print predictions_as_dataframe

         id                                        ingredients Prediction DT
0     18009  [baking powder, eggs, all-purpose flour, raisi...          thai
1     28583  [sugar, egg yolks, corn starch, cream of tarta...  cajun_creole
2     41580  [sausage links, fennel bulb, fronds, olive oil...  cajun_creole
3     29752  [meat cuts, file powder, smoked sausage, okra,...      filipino
4     35687  [ground black pepper, salt, sausage casings, l...        indian
5     38527  [baking powder, all-purpose flour, peach slice...        korean
6     19666             [grape juice, orange, white zinfandel]        korean
7     41217  [ground ginger, white pepper, green onions, or...        korean
8     28753  [diced onions, taco seasoning mix, all-purpose...        korean
9     22659  [eggs, cherries, dates, dark muscovado sugar, ...    vietnamese
10    21749  [pasta, olive oil, crushed red pepper, cherry ...        korean
11    44967  [water, butter, ground sumac, ground lamb, gro...       italian

In [31]:
#Predict on test set and write the result out.
final_result=clf.predict(build_designmatrix(test_df))
test_df['cuisine']=final_result
test_df.to_csv("Decision Tree Submission.csv")

### Random Forest Classification Model

A random forest is a model ensemble. An ensemble combines multiple models to achieve better results than a single model would. A random forest consists of multiple decision trees. Each tree in the forest has a different random subset of the features (subspace sampling) and the trees are fed with different subsets of the training data (bagging). Though a random forest gives more accurate results than a single decision tree, it is harder to read and takes more computational time to generate.



In [13]:
from time import time

# Machine Learning
from sklearn.ensemble import RandomForestClassifier

# Helper
from sklearn.cross_validation import cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer

In [14]:
words = [' '.join(item) for item in full_df.ingredients]

In [16]:
vec = CountVectorizer(max_features=2000)  #Convert a collection of text documents to a matrix of token counts
bag_of_words = vec.fit(words).transform(words).toarray() #Transform documents to document-term matrix.
len(bag_of_words)

39774

In [17]:
random_forest = RandomForestClassifier(n_estimators=200) #creates a 200 tree forest

In [20]:
%%time
start = time()
random_forest.fit(bag_of_words, full_df.cuisine) #Learn a vocabulary dictionary of all tokens in the raw documents

CPU times: user 3min 12s, sys: 2.02 s, total: 3min 14s
Wall time: 3min 14s


In [22]:
%%time
start = time()
train_pred = cross_val_predict(random_forest, bag_of_words, full_df.cuisine, cv=2) #Generate cross-validated estimates for each input data point

CPU times: user 2min 38s, sys: 3.36 s, total: 2min 41s
Wall time: 2min 42s


In [23]:
len(train_pred)

39774

In [26]:
print("Accuracy: ", accuracy_score(full_df.cuisine, train_pred))

('Accuracy: ', 0.74108714235430184)


In [27]:
test = pd.read_json('test.json')

In [28]:
test_words = [' '.join(item) for item in test.ingredients]
test_bag = vec.transform(test_words).toarray()

In [29]:
len(test_bag)

9944

In [30]:
result = random_forest.predict(test_bag)

output = pd.DataFrame(data={"id":test.id, "ingredients":test.ingredients, "cuisine":result})
output.head(20)
output.to_csv("Random Forest Submission.csv")

### Ridge Regression Model

In [35]:
%%time
from sklearn import preprocessing
from sklearn.linear_model import Ridge
est = Ridge(alpha=10)

labels = train_df['cuisine']
le = preprocessing.LabelEncoder()
labels_fea = le.fit_transform(labels)
est_fit=est.fit(build_designmatrix(train_df), labels_fea)
est_predict=est_fit.predict(build_designmatrix(test_df))
labels_converted = le.inverse_transform(est_predict.astype('I'))



IndexError: index 28 is out of bounds for axis 0 with size 20

In [32]:
predictions_as_dataframe = test_df.join(pd.DataFrame({"Prediction R": labels_converted}))
print predictions_as_dataframe

ValueError: LabelEncoder was not fitted yet.

### Naive Bayes Model

In [36]:
from sklearn.feature_extraction.text import CountVectorizer

def conc_str(in_list):
    return ','.join(in_list)

def make_xy(indf, vectorizer=None):
    #Your code here    
    if vectorizer is None:
        vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(indf['ingredients'].apply(conc_str))
    X = X.tocsc()  # some versions of sklearn return COO format
    y = indf['cuisine']
    return X, y

X, y = make_xy(full_df)

In [37]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X, y)
clf = MultinomialNB().fit(xtrain, ytrain)
print "MN Accuracy: %0.2f%%" % (100 * clf.score(xtest, ytest))

MN Accuracy: 72.21%


In [38]:
training_accuracy = clf.score(xtrain, ytrain)
test_accuracy = clf.score(xtest, ytest)

print "Accuracy on training data: %0.2f" % (training_accuracy)
print "Accuracy on test data:     %0.2f" % (test_accuracy)

Accuracy on training data: 0.75
Accuracy on test data:     0.72


In [39]:
from sklearn.cross_validation import KFold
def cv_score(clf, X, y, scorefunc):
    result = 0.
    nfold = 5
    for train, test in KFold(y.size, nfold): # split data into train/test groups, 5 times
        clf.fit(X[train], y[train]) # fit
        result += scorefunc(clf, X[test], y[test]) # evaluate score function on held-out data
    return result / nfold # average

In [40]:
def log_likelihood(clf, x, y):
    prob = clf.predict_log_proba(x)
    rotten = y == 0
    fresh = ~rotten
    return prob[rotten, 0].sum() + prob[fresh, 1].sum()

In [46]:
from sklearn.cross_validation import train_test_split
itrain, itest = train_test_split(xrange(full_df.shape[0]), train_size=0.7)
mask=np.ones(full_df.shape[0], dtype='int')
mask[itrain]=1
mask[itest]=0
mask = (mask==1)

Xtrain, Xtest, ytrain, ytest = X[mask], X[~mask], y[mask], y[~mask]

In [47]:
#the grid of parameters to search over
alphas = [0, .1, 1, 5, 10, 50]
min_dfs = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1]

#Find the best value for alpha and min_df, and the best classifier
best_alpha = None
best_min_df = None
maxscore=-np.inf
for alpha in alphas:
    for min_df in min_dfs:         
        vectorizer = CountVectorizer(min_df = min_df)       
        Xthis, ythis = make_xy(full_df, vectorizer)
        Xtrainthis=Xthis[mask]
        ytrainthis=ythis[mask]
        #your code here
        clf = MultinomialNB(alpha=alpha)
        cvscore = cv_score(clf, Xtrainthis, ytrainthis, log_likelihood)

        if cvscore > maxscore:
            maxscore = cvscore
            best_alpha, best_min_df = alpha, min_df

ValueError: Unknown label type: array([nan, u'mexican', u'korean', ..., nan, nan, u'british'], dtype=object)

### Ensemble Model

In [48]:
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - #
# Generic classification and optimization functions from CS-109 labs
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - #

from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - #
# clf - original classifier
# parameters - grid to search over
# X - usually your training X matrix
# y - usually your training y 
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - #
def cv_optimize(clf, parameters, X, y, n_jobs=1, n_folds=5, score_func=None):
    if score_func:
        gs = GridSearchCV(clf, param_grid=parameters, cv=n_folds, n_jobs=n_jobs, scoring=score_func)
    else:
        gs = GridSearchCV(clf, param_grid=parameters, n_jobs=n_jobs, cv=n_folds)
    gs.fit(X, y)
    print "BEST", gs.best_params_, gs.best_score_, gs.grid_scores_
    best = gs.best_estimator_
    return best

In [49]:
%%time
from sklearn import tree
clfTree1 = tree.DecisionTreeClassifier()
#parameters = {"max_depth": [1, 2, 3, 4, 5, 6, 7], 'min_samples_leaf': [1, 2, 3, 4, 5, 6]}
#parameters = {"max_depth": [1, 2, 3, 4, 5, 6, 7], 'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}
parameters = {"max_depth": [None], 'min_samples_leaf': [4, 5, 6]}

clf = cv_optimize(clfTree1, parameters, Xtrain, ytrain, n_jobs=1, n_folds=5, score_func='f1_weighted')
clf=clf.fit(Xtrain, ytrain)
    
training_accuracy = clf.score(Xtrain, ytrain)
test_accuracy = clf.score(Xtest, ytest)
print "############# based on standard predict ################"
print "Accuracy on training data: %0.2f" % (training_accuracy)
print "Accuracy on test data:     %0.2f" % (test_accuracy)
print confusion_matrix(ytest, clf.predict(Xtest))
print "########################################################"

BEST {'max_depth': None, 'min_samples_leaf': 5} 0.61568932745 [mean: 0.61375, std: 0.00609, params: {'max_depth': None, 'min_samples_leaf': 4}, mean: 0.61569, std: 0.00569, params: {'max_depth': None, 'min_samples_leaf': 5}, mean: 0.61451, std: 0.00568, params: {'max_depth': None, 'min_samples_leaf': 6}]
############# based on standard predict ################
Accuracy on training data: 0.76
Accuracy on test data:     0.63
[[  53    2    7    1    2    8    1    3    1   16    1    0    0   10
     2    2   15    6    0    6]
 [   2   61    2    1    4   45    3    5   15   25    4    0    1    5
     2   12   36    1    0    0]
 [   4    9  262    6    3   37    1    9    3   39    8    2    0   28
     2    2   64    4    0    0]
 [   1    8    7  602   27    9    1    9    0   17    1   50   24   11
     0    4   23    1   19   12]
 [   8    5    5   40   73   10    2    3    1    8    5    6    6    7
     0    0   14    5    7    3]
 [   4   28   13   10    2  349   16    7   12  

In [ ]:
%%time

from sklearn.ensemble import RandomForestClassifier
clfForest = RandomForestClassifier()
parameters = {"n_estimators": range(1, 20)}

clf = cv_optimize(clfForest, parameters, Xtrain, ytrain, n_jobs=1, n_folds=5, score_func='f1_weighted')
clf=clf.fit(Xtrain, ytrain)

training_accuracy = clf.score(Xtrain, ytrain)
test_accuracy = clf.score(Xtest, ytest)
print "############# based on standard predict ################"
print "Accuracy on training data: %0.2f" % (training_accuracy)
print "Accuracy on test data:     %0.2f" % (test_accuracy)
print confusion_matrix(ytest, clf.predict(Xtest))
print "########################################################"

In [ ]:
%%time

from sklearn.ensemble import AdaBoostClassifier
clfAda = AdaBoostClassifier()
parameters = {"n_estimators": range(10, 60)}

clf = cv_optimize(clfAda, parameters, Xtrain, ytrain, n_jobs=1, n_folds=5, score_func='f1_weighted')
clf=clf.fit(Xtrain, ytrain)

training_accuracy = clf.score(Xtrain, ytrain)
test_accuracy = clf.score(Xtest, ytest)
print "############# based on standard predict ################"
print "Accuracy on training data: %0.2f" % (training_accuracy)
print "Accuracy on test data:     %0.2f" % (test_accuracy)
print confusion_matrix(ytest, clf.predict(Xtest))
print "########################################################"

## Visualization

### Decision Tree Graph

In [ ]:
from sklearn.externals.six import StringIO  
import pydot 
dot_data = StringIO() 
tree.export_graphviz(clf, out_file=dot_data) 
graph = pydot.graph_from_dot_data(dot_data.getvalue()) 
graph.write_pdf("Decision Tree Graph.pdf") 

In [ ]:
from IPython.display import Image  
dot_data = StringIO()  
tree.export_graphviz(clf, out_file=dot_data,  
                         feature_names=iris.feature_names,  
                         class_names=iris.target_names,  
                         filled=True, rounded=True,  
                         special_characters=True)  
graph = pydot.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())  

### Decision surface of Decision tree

In [9]:
%%time
from sklearn import tree

clf = tree.DecisionTreeClassifier()
clf = clf.fit(build_designmatrix(train_df), train_df['cuisine'])
clf_predict=clf.predict(build_designmatrix(validation_df))

CPU times: user 5min 19s, sys: 3.53 s, total: 5min 23s
Wall time: 5min 24s


In [32]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

# Parameters
n_classes = 3
plot_colors = "bry"
plot_step = 0.02

for pairidx, pair in enumerate([[0, 1], [0, 2], [0, 3],
                                [1, 2], [1, 3], [2, 3]]):
    # We only take the two corresponding features
    
    trainMatrix = build_designmatrix(train_df)
    X = trainMatrix[:, pair]
    y = train_df['cuisine']

    # Shuffle
    idx = np.arange(X.shape[0])
    np.random.seed(13)
    np.random.shuffle(idx)
    X = X[idx]
    y = y[idx]

    # Standardize
    mean = X.mean(axis=0)
    std = X.std(axis=0)
    X = (X - mean) / std

    # Train
    clf = DecisionTreeClassifier().fit(X, y)

    # Plot the decision boundary
    plt.subplot(2, 3, pairidx + 1)

    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, plot_step),
                         np.arange(y_min, y_max, plot_step))

    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    cs = plt.contourf(xx, yy, Z, cmap=plt.cm.Paired)

    plt.xlabel(trainMatrix.feature_names[pair[0]])
    plt.ylabel(trainMatrix.feature_names[pair[1]])
    plt.axis("tight")

    # Plot the training points
    for i, color in zip(range(n_classes), plot_colors):
        idx = np.where(y == i)
        plt.scatter(X[idx, 0], X[idx, 1], c=color, label=trainMatrix.target_names[i],
                    cmap=plt.cm.Paired)

    plt.axis("tight")

plt.suptitle("Decision surface of a decision tree using paired features")
plt.legend()
plt.show()

ValueError: could not convert string to float: italian

### Decision Tree Regression

In [ ]:
# Import the necessary modules and libraries
import numpy as np
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt

# Create a random dataset
rng = np.random.RandomState(1)
X = np.sort(5 * rng.rand(80, 1), axis=0)
y = np.sin(X).ravel()
y[::5] += 3 * (0.5 - rng.rand(16))

# Fit regression model
regr_1 = DecisionTreeRegressor(max_depth=2)
regr_2 = DecisionTreeRegressor(max_depth=5)
regr_1.fit(X, y)
regr_2.fit(X, y)

# Predict
X_test = np.arange(0.0, 5.0, 0.01)[:, np.newaxis]
y_1 = regr_1.predict(X_test)
y_2 = regr_2.predict(X_test)

# Plot the results
plt.figure()
plt.scatter(X, y, c="k", label="data")
plt.plot(X_test, y_1, c="g", label="max_depth=2", linewidth=2)
plt.plot(X_test, y_2, c="r", label="max_depth=5", linewidth=2)
plt.xlabel("data")
plt.ylabel("target")
plt.title("Decision Tree Regression")
plt.legend()
plt.show()